In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as dt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier


In [2]:
file_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
full_path = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'train_timeseries.csv')


In [3]:
full_path

'/Users/voxxy/code/realGhostFoxx/droughts_modelling/raw_data/train_timeseries.csv'

In [70]:
train = pd.read_csv(full_path)

In [71]:
train.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,9.50,14.65,2.20,2.94,1.49,1.46,4.85,6.04,3.23,2.81,NaN
1,1001,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,10.18,16.60,2.52,3.43,1.83,1.60,5.33,6.13,3.72,2.41,NaN
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,7.41,18.41,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,15.92,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,13.48,2.65,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN


In [72]:
train['week_num'] = pd.to_datetime(train['date']).dt.isocalendar().week

In [73]:
train['year'] = pd.to_datetime(train['date']).dt.year

In [74]:
train.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score,week_num,year
0,1001,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,9.50,14.65,2.20,2.94,1.49,1.46,4.85,6.04,3.23,2.81,NaN,52,2000
1,1001,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,10.18,16.60,2.52,3.43,1.83,1.60,5.33,6.13,3.72,2.41,NaN,52,2000
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,7.41,18.41,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN,1,2000
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,15.92,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0,1,2000
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,13.48,2.65,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN,1,2000


In [75]:
train.groupby(['fips', 'year', 'week_num']).agg({'PRECTOT': ['mean'], 'PS': ['mean'], 'QV2M': ['mean'],
                'T2M': ['mean'], 'T2MDEW': ['mean'], 'T2MWET': ['mean'], 'T2M_MAX': ['mean'], 'T2M_MIN': ['mean'],
                'T2M_RANGE': ['mean'], 'TS': ['mean'], 'WS10M': ['mean'], 'WS10M_MAX': ['mean'],
                    'WS10M_MIN': ['mean'], 'WS10M_RANGE': ['mean'], 'WS50M': ['mean'],'WS50M_MAX': ['mean'],
                        'WS50M_MIN': ['mean'], 'WS50M_RANGE': ['mean'], 'score': 'max'}).reset_index().sort_values(['fips', 'year', 'week_num'])

,fips,year,week_num,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
,,,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,max
0,1001,2000,1,5.261429,100.702857,6.491429,9.878571,5.648571,5.674286,16.345714,3.647143,12.698571,9.397143,2.630000,3.722857,1.520000,2.201429,5.044286,6.934286,2.815714,4.120000,1.0
1,1001,2000,2,3.554286,101.100000,6.104286,9.662857,5.527143,5.552857,16.751429,2.928571,13.825714,9.431429,2.472857,3.585714,1.568571,2.018571,5.142857,7.162857,2.968571,4.192857,2.0
2,1001,2000,3,4.017143,100.348571,6.090000,8.270000,4.834286,4.890000,13.738571,2.795714,10.941429,8.194286,2.507143,3.574286,1.461429,2.110000,4.702857,6.682857,2.997143,3.685714,2.0
3,1001,2000,4,3.742857,100.755714,3.072857,0.737143,-3.367143,-3.265714,5.570000,-3.320000,8.890000,0.817143,2.774286,3.908571,1.664286,2.248571,4.928571,6.727143,3.244286,3.480000,2.0
4,1001,2000,5,0.000000,101.012857,3.142857,2.501429,-2.685714,-2.627143,10.460000,-3.842857,14.301429,2.021429,2.211429,3.161429,1.225714,1.934286,4.125714,5.971429,2.045714,3.927143,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766115,56043,2016,49,0.624286,82.787143,1.841429,-8.072857,-12.847143,-12.498571,-3.115714,-12.475714,9.361429,-8.557143,3.268571,5.190000,1.485714,3.704286,4.784286,7.161429,2.328571,4.831429,0.0
2766116,56043,2016,50,0.754286,82.602857,1.574286,-10.298571,-14.704286,-14.268571,-4.091429,-15.560000,11.467143,-11.231429,3.212857,5.612857,1.051429,4.561429,4.600000,7.524286,1.565714,5.957143,0.0
2766117,56043,2016,51,0.775714,82.742857,2.201429,-5.717143,-9.964286,-9.821429,0.598571,-11.361429,11.960000,-7.302857,4.641429,6.664286,2.394286,4.268571,6.905714,9.538571,4.108571,5.428571,0.0


In [77]:
train.columns


Index(['fips', 'date', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
       'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'score', 'week_num', 'year'],
      dtype='object')

In [64]:
train.columns = ['_'.join(col) for col in train.columns.values]
aggregated_df.columns = ['_'.join(col) for col in slim_origin_1001.columns.values]
# train['score_max'] = train['score_max'].map(lambda x: np.round(x))

In [65]:
train.head()

,f_i_p_s,d_a_t_e,P_R_E_C_T_O_T,P_S,Q_V_2_M,T_2_M,T_2_M_D_E_W,T_2_M_W_E_T,T_2_M___M_A_X,T_2_M___M_I_N,T_2_M___R_A_N_G_E,T_S,W_S_1_0_M,W_S_1_0_M___M_A_X,W_S_1_0_M___M_I_N,W_S_1_0_M___R_A_N_G_E,W_S_5_0_M,W_S_5_0_M___M_A_X,W_S_5_0_M___M_I_N,W_S_5_0_M___R_A_N_G_E,s_c_o_r_e,w_e_e_k___n_u_m,y_e_a_r
0,1001,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,9.50,14.65,2.20,2.94,1.49,1.46,4.85,6.04,3.23,2.81,NaN,52,2000
1,1001,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,10.18,16.60,2.52,3.43,1.83,1.60,5.33,6.13,3.72,2.41,NaN,52,2000
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,7.41,18.41,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN,1,2000
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,15.92,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0,1,2000
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,13.48,2.65,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN,1,2000


In [6]:
def weekly_aggregate():
    # train = self.data
    train = pd.read_csv(full_path)
    
    train['week_num'] = pd.to_datetime(train['date']).dt.isocalendar().week
    train['year'] = pd.to_datetime(train['date']).dt.year
    
    train = train.groupby(['fips', 'year', 'week_num']).agg({'PRECTOT': ['mean'], 'PS': ['mean'], 'QV2M': ['mean'],
                'T2M': ['mean'], 'T2MDEW': ['mean'], 'T2MWET': ['mean'], 'T2M_MAX': ['mean'], 'T2M_MIN': ['mean'],
                'T2M_RANGE': ['mean'], 'TS': ['mean'], 'WS10M': ['mean'], 'WS10M_MAX': ['mean'],
                    'WS10M_MIN': ['mean'], 'WS10M_RANGE': ['mean'], 'WS50M': ['mean'],'WS50M_MAX': ['mean'],
                        'WS50M_MIN': ['mean'], 'WS50M_RANGE': ['mean'], 'score': 'max'}).reset_index().sort_values(['fips', 'year', 'week_num'])
    
    return train.dropna()


In [7]:
weekly_aggregate()

,fips,year,week_num,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
,,,,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,mean,max
0,1001,2000,1,5.261429,100.702857,6.491429,9.878571,5.648571,5.674286,16.345714,...,9.397143,2.630000,3.722857,1.520000,2.201429,5.044286,6.934286,2.815714,4.120000,1.0
1,1001,2000,2,3.554286,101.100000,6.104286,9.662857,5.527143,5.552857,16.751429,...,9.431429,2.472857,3.585714,1.568571,2.018571,5.142857,7.162857,2.968571,4.192857,2.0
2,1001,2000,3,4.017143,100.348571,6.090000,8.270000,4.834286,4.890000,13.738571,...,8.194286,2.507143,3.574286,1.461429,2.110000,4.702857,6.682857,2.997143,3.685714,2.0
3,1001,2000,4,3.742857,100.755714,3.072857,0.737143,-3.367143,-3.265714,5.570000,...,0.817143,2.774286,3.908571,1.664286,2.248571,4.928571,6.727143,3.244286,3.480000,2.0
4,1001,2000,5,0.000000,101.012857,3.142857,2.501429,-2.685714,-2.627143,10.460000,...,2.021429,2.211429,3.161429,1.225714,1.934286,4.125714,5.971429,2.045714,3.927143,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766114,56043,2016,48,0.211429,82.517143,2.265714,-3.541429,-9.130000,-9.007143,0.507143,...,-4.108571,4.227143,6.260000,2.197143,4.057143,6.231429,8.827143,3.687143,5.140000,0.0
2766115,56043,2016,49,0.624286,82.787143,1.841429,-8.072857,-12.847143,-12.498571,-3.115714,...,-8.557143,3.268571,5.190000,1.485714,3.704286,4.784286,7.161429,2.328571,4.831429,0.0
2766116,56043,2016,50,0.754286,82.602857,1.574286,-10.298571,-14.704286,-14.268571,-4.091429,...,-11.231429,3.212857,5.612857,1.051429,4.561429,4.600000,7.524286,1.565714,5.957143,0.0
